In [1]:
from curriculum import get_exam_curriculum, get_rubrics_by_sub_strand
from utils import *
import random

## Generate questions

In [2]:
bloom_skill_count = 3

In [3]:
question_brd = get_exam_curriculum(
    strand_ids=[1,2,3,4,5,6,7,8,9,10], 
    question_count=25,
    is_debug=True, 
    curriculum_file="data/cbc_data.json",
    bloom_skill_count=bloom_skill_count,
)

✅ Question breakdown written to output/question_breakdown.json


In [4]:
# parsed_output = generate_llm_question_list(
#     llm=OPENAI_LLM_4O,
#     input_data=question_brd,
#     bloom_skill_count=bloom_skill_count,
#     is_debug=True,
# )
parsed_output = [
    {
        "questions": "Amina notices that her phone helps her communicate with friends and learn new things. How does science play a role in making her phone useful in daily life?",
        "expected_answers": "Science contributes to the development of technology like phones, which help in communication, learning, and accessing information."
    },
    {
        "questions": "Brian has a mixture of salt and water. Describe a method he can use to separate the salt from the water.",
        "expected_answers": "Brian can use evaporation to separate the salt from the water. By heating the mixture, the water will evaporate, leaving the salt behind."
    },
    {
        "questions": "Zawadi is learning about the human excretory system. Explain the function of the kidneys in this system.",
        "expected_answers": "The kidneys filter waste products and excess water from the blood to form urine, which is then excreted from the body."
    },
    {
        "questions": "Musa wants to hang a poster on his metal locker using a magnet. Explain why a magnet is suitable for this task.",
        "expected_answers": "A magnet is suitable because it can attract and hold onto the metal surface of the locker, keeping the poster in place without damaging it."
    }
]


In [5]:
QUESTION_LIST_OUTPUT_FILE = "output/question_list.json"

with open(QUESTION_LIST_OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(parsed_output, f, ensure_ascii=False, indent=4)

print(f"✅ Question list written to {QUESTION_LIST_OUTPUT_FILE}")

✅ Question list written to output/question_list.json


In [6]:
def convert_to_single_qa_list(data):
    result = []
    for index, item in enumerate(data):
        questions = item["questions"]
        expected_answers = item["expected_answers"]

        if isinstance(questions, str):
            questions = [questions]
        if isinstance(expected_answers, str):
            expected_answers = [expected_answers]

        if questions and expected_answers:
            result.append({
                "id": index + 1,
                "question": questions[0],
                "expected_answer": expected_answers[0]
            })

    return result

exam_output = convert_to_single_qa_list(parsed_output)
print(exam_output)

[{'id': 1, 'question': 'Amina notices that her phone helps her communicate with friends and learn new things. How does science play a role in making her phone useful in daily life?', 'expected_answer': 'Science contributes to the development of technology like phones, which help in communication, learning, and accessing information.'}, {'id': 2, 'question': 'Brian has a mixture of salt and water. Describe a method he can use to separate the salt from the water.', 'expected_answer': 'Brian can use evaporation to separate the salt from the water. By heating the mixture, the water will evaporate, leaving the salt behind.'}, {'id': 3, 'question': 'Zawadi is learning about the human excretory system. Explain the function of the kidneys in this system.', 'expected_answer': 'The kidneys filter waste products and excess water from the blood to form urine, which is then excreted from the body.'}, {'id': 4, 'question': 'Musa wants to hang a poster on his metal locker using a magnet. Explain why 

## Simulate student answers

In [7]:
def generate_random_scores(count):
    return [{"id": i+1, "avg_score": random.randint(0, 100)} for i in range(count)]

# sample_students = generate_random_scores(4)
sample_students = [{'id': 1, 'avg_score': 12}, {'id': 2, 'avg_score': 95}, {'id': 3, 'avg_score': 54}, {'id': 4, 'avg_score': 71}]
print(sample_students)

[{'id': 1, 'avg_score': 12}, {'id': 2, 'avg_score': 95}, {'id': 3, 'avg_score': 54}, {'id': 4, 'avg_score': 71}]


In [8]:
# parsed_answers = generate_llm_exam_answers_list(
#     llm=OPENAI_LLM_4O,
#     exam_data=exam_output,
#     student_data=sample_students,
#     is_debug=True,
# )
parsed_answers = [
    {
        "id": 1,
        "answers": [
            {
                "question_id": "1",
                "answer": "Phones are made using science, so they help us talk to friends and learn."
            },
            {
                "question_id": "2",
                "answer": "Maybe he can just let the water dry up and the salt will stay."
            },
            {
                "question_id": "3",
                "answer": "Kidneys clean the blood and make urine."
            },
            {
                "question_id": "4",
                "answer": "Magnets stick to metal, so it can hold the poster."
            }
        ]
    },
    {
        "id": 2,
        "answers": [
            {
                "question_id": "1",
                "answer": "Science is used to create the technology in phones, allowing us to communicate and learn easily."
            },
            {
                "question_id": "2",
                "answer": "Brian can heat the mixture so the water evaporates, leaving the salt behind."
            },
            {
                "question_id": "3",
                "answer": "The kidneys filter out waste and extra water from the blood, creating urine to be excreted."
            },
            {
                "question_id": "4",
                "answer": "A magnet is suitable because it attracts the metal locker, holding the poster without damage."
            }
        ]
    },
    {
        "id": 3,
        "answers": [
            {
                "question_id": "1",
                "answer": "Science helps make phones that we use to talk and learn."
            },
            {
                "question_id": "2",
                "answer": "He can heat the water so it evaporates and leaves the salt."
            },
            {
                "question_id": "3",
                "answer": "Kidneys take out waste from blood and make urine."
            },
            {
                "question_id": "4",
                "answer": "Magnets can stick to metal, so they hold the poster."
            }
        ]
    },
    {
        "id": 4,
        "answers": [
            {
                "question_id": "1",
                "answer": "Science helps in making phones that we use for communication and learning."
            },
            {
                "question_id": "2",
                "answer": "He can use evaporation by heating the mixture to get the salt."
            },
            {
                "question_id": "3",
                "answer": "The kidneys filter waste from the blood and make urine."
            },
            {
                "question_id": "4",
                "answer": "A magnet can hold the poster because it sticks to metal."
            }
        ]
    }
]

In [9]:
# # ==== DB MOCK ANSWERS
# # ====================
# exam = []
# EXAM_FILE = "data/exam.json"

# with open(EXAM_FILE, "r") as f:
#     exam = json.load(f)

# print(exam)


# students = []
# STUDENT_FILE = "data/classroom.json"

# with open(STUDENT_FILE, "r") as f:
#     students = json.load(f)

# print(students)


# parsed_answers = generate_llm_exam_answers_list(
#     llm=OPENAI_LLM_4O,
#     exam_data=exam,
#     student_data=students,
#     is_debug=True,
# )

In [10]:
ANSWERS_LIST_OUTPUT_FILE = "output/answers_list.json"

with open(ANSWERS_LIST_OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(parsed_answers, f, ensure_ascii=False, indent=4)

print(f"✅ Answers list written to {ANSWERS_LIST_OUTPUT_FILE}")

✅ Answers list written to output/answers_list.json


## Grade answers

In [11]:
exam_answers = []
EXAM_ANSWERS_FILE = "data/answers.json"

with open(EXAM_ANSWERS_FILE, "r") as f:
    exam_answers = json.load(f)

print(exam_answers)

[{'question_id': 51, 'question_description': 'Amina noticed a broken glass beaker on the laboratory floor. What should she do to ensure safety?', 'expected_answer': 'Amina should inform the teacher immediately and ensure no one steps on the broken glass.', 'sub_strand': 'Laboratory Safety', 'answers': [{'answer_id': 1, 'description': 'Amina should tell the teacher right away and make sure no one steps on the broken glass.'}, {'answer_id': 26, 'description': 'Amina should tell the teacher and make sure no one steps on it.'}, {'answer_id': 51, 'description': 'Amina should tell the teacher.'}, {'answer_id': 76, 'description': 'Amina should inform the teacher immediately and make sure no one steps on the glass.'}, {'answer_id': 101, 'description': 'Amina should tell the teacher and make sure no one steps on the glass.'}, {'answer_id': 126, 'description': 'Amina should tell the teacher and make sure no one steps on it.'}, {'answer_id': 151, 'description': 'Amina should tell the teacher and 

In [12]:
updated_exam_answers = []
for qa in exam_answers:
    qa['rubrics'] = sub_strand_rubrics = get_rubrics_by_sub_strand(
            sub_strand_name=qa['sub_strand'],
            curriculum_file="data/cbc_data.json",
        )
    updated_exam_answers.append(qa)

In [13]:
EXAM_ANSWERS_FILE = "data/answers.json"

with open(EXAM_ANSWERS_FILE, 'w', encoding='utf-8') as f:
    json.dump(updated_exam_answers, f, ensure_ascii=False, indent=4)

print(f"✅ Updated exam answers written to {EXAM_ANSWERS_FILE}")

✅ Updated exam answers written to data/answers.json


In [20]:
parsed_grades = generate_llm_answer_grading_list(
    llm=OPENAI_LLM_4O,
    question_data=updated_exam_answers[6],
    is_debug=True,
)

📝 Input token count (gpt-4o): 776
📦 Raw LLM output:
 content='```json\n[\n  {\n    "answer_id": "7",\n    "score": "4"\n  },\n  {\n    "answer_id": "32",\n    "score": "3"\n  },\n  {\n    "answer_id": "57",\n    "score": "2"\n  },\n  {\n    "answer_id": "82",\n    "score": "4"\n  },\n  {\n    "answer_id": "107",\n    "score": "3"\n  },\n  {\n    "answer_id": "132",\n    "score": "3"\n  },\n  {\n    "answer_id": "157",\n    "score": "3"\n  },\n  {\n    "answer_id": "182",\n    "score": "3"\n  },\n  {\n    "answer_id": "207",\n    "score": "3"\n  },\n  {\n    "answer_id": "232",\n    "score": "3"\n  }\n]\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 197, 'prompt_tokens': 783, 'total_tokens': 980, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'sy

In [21]:
GRADES_LIST_OUTPUT_FILE = "output/grades_list.json"

with open(GRADES_LIST_OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(parsed_grades, f, ensure_ascii=False, indent=4)

print(f"✅ Grades list written to {GRADES_LIST_OUTPUT_FILE}")

✅ Grades list written to output/grades_list.json
